In [1]:
# --- 理論計算 ---

# 工作站的平行機台數（與表格相同順序）
p = [3, 3, 2, 5, 5]  

# 工作站的單件加工時間 (小時/件)
t = [5, 8, 6, 8, 12]

# 各站產能 TH_i = p_i / t_i
TH_stations = [p[i] / t[i] for i in range(len(p))]

# 瓶頸產能 (系統最大長期產出率)
r_b = min(TH_stations)

# 純加工時間總和
T0 = sum(t)

# 關鍵在製品量
W0 = r_b * T0

print("=== 理論計算結果 ===")
print(f"各站產能 TH_i: {['%.3f'%(th) for th in TH_stations]} (件/小時)")
print(f"瓶頸產能 r_b = {r_b:.3f} (件/小時)")
print(f"T0 (純加工時間總和) = {T0:.1f} 小時")
print(f"W0 (關鍵在製品量) = {W0:.2f}\n")

=== 理論計算結果 ===
各站產能 TH_i: ['0.600', '0.375', '0.333', '0.625', '0.417'] (件/小時)
瓶頸產能 r_b = 0.333 (件/小時)
T0 (純加工時間總和) = 39.0 小時
W0 (關鍵在製品量) = 13.00



In [ ]:
import simpy
import statistics

def product(env, name, stations, processing_times, cycle_times, completions):
    """
    模擬一個產品在生產線上的循環過程。
    產品必須依序通過所有工作站，每站加工時間由 processing_times 指定。
    每完成一個循環，就記錄其週期時間。
    """
    while True:
        start_time = env.now  # 紀錄進入第一站的時間
        # 依序通過每個工作站
        for i, station in enumerate(stations):
            with station.request() as req:
                yield req
                # 模擬加工所需時間
                yield env.timeout(processing_times[i])
        # 完成所有站後，計算該產品一個循環的生產週期時間
        ct = env.now - start_time
        cycle_times.append(ct)
        completions.append(env.now)
        # 模擬閉環生產：產品完成後，立即重新進入生產線
        # 可視為一個固定在製品的閉環系統

def run_simulation(WIP, processing_times, sim_time):
    """
    建立一個閉環生產線模擬，固定在製品數為 WIP，
    processing_times 為各工作站的加工時間，模擬時間為 sim_time。
    回傳模擬期間的產出率、平均週期時間以及完成循環次數。
    """
    env = simpy.Environment()
    # 建立每個工作站資源，均只有一台機器
    stations = [simpy.Resource(env, capacity=1) for _ in processing_times]
    cycle_times = []   # 用來記錄每個產品一個循環的週期時間
    completions = []   # 用來記錄每次完成循環的時間點

    # 初始時產生 WIP 個產品（閉環系統，每個產品持續循環）
    for i in range(WIP):
        env.process(product(env, f'Product {i+1}', stations, processing_times, cycle_times, completions))
    
    # 執行模擬
    env.run(until=sim_time)
    
    # 計算產出率：完成循環的數量 / 模擬總時間
    throughput = len(completions) / sim_time
    avg_cycle_time = statistics.mean(cycle_times) if cycle_times else None
    return throughput, avg_cycle_time, len(cycle_times)

# 模擬參數設定
processing_times = [2, 3, 4]  # 各工作站加工時間（秒）
T0 = sum(processing_times)    # 理論最小週期時間 = 9 秒
r_b = 1 / max(processing_times)  # 瓶頸速率 = 0.25  件/秒
W0 = r_b * T0                 # 臨界在製品量 = 2.25，實際上至少 3 個

sim_time = 1000  # 模擬時間（秒）

print("理論參數:")
print(f"總加工時間 T0 = {T0} 秒")
print(f"瓶頸速率 r_b = {r_b:.3f} (件/秒)")
print(f"臨界在製品量 W0 ≈ {W0:.2f} (實際至少為 3 個)\n")

# 測試不同在製品數量下的模擬結果
for WIP in [2, 3, 4, 5]:
    throughput, avg_cycle_time, num_cycles = run_simulation(WIP, processing_times, sim_time)
    print(f"WIP = {WIP}:")
    print(f"  產出率 (Throughput) = {throughput:.3f} 件/秒")
    print(f"  平均週期時間 (Cycle Time) = {avg_cycle_time:.3f} 秒")
    print(f"  完成循環次數 = {num_cycles}\n")